In [1]:
import os

In [2]:
%pwd

'd:\\MLOPS\\End-to-end-Chest-Cancer-Classification-using-MLfolow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\MLOPS\\End-to-end-Chest-Cancer-Classification-using-MLfolow-DVC'

In [ ]:
# use dagshub, mlflow for model tracking
# connect github repo with dagshub repo 
# after doing all set up,click on Eperimenets and copy link.


In [ ]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/...repolink.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "yadav"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "password"

In [ ]:
MLFLOW_TRACKING_URI = https://dagshub.com/....repolink.mlflow \
MLFLOW_TRACKING_USERNAME = yadav \
MLFLOW_TRACKING_PASSWORD = password \
python script.py

In [5]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model("artifacts/training/model.h5") 

In [7]:
# config.yaml

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size : int

In [ ]:
# model configuration manager
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data= "artifacts/data_ingestion/chest-ct-scan-image",
            mlflow_uri= "https://dagshub.com/....repo.mlflow",
            all_params = self.params,
            params_image = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config
    

In [10]:
# components
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
# we can get below code from keras website
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_genrator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30 
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation  = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def  evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path = Path("scores.json"),data = scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depend on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(
                    self.model, "model", registered_model_name = "VGG16Model")                
            else:
                mlflow.keras.log_model(self.model, "model")
                




In [ ]:
# after running above experiements, we will get scores.json file
# we can see our model in dagshub mlflow model registry

In [ ]:
try: 
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

In [ ]:
# now we can see our model in dagshub mlflow model registry
# also we can play with different parameteres in params.yaml file
# rerun whole pipeline